## Introduction 
In this lab, we will see how to take a video and run it through Amazon Rekognition Video to extract the following:

1. Text - Find text in the video
1. Content Moderation Labels - Is the video suitable for viewing; rate the video for violence, gore, nudity, etc.
1. Labels - Find labels in the video, including objects (tree), events (birthday), activities (skiing)

The following AWS services are used in this lab:

* **IAM** : Identity and Access Management, mostly for Permissions and Roles  
* **Amazon Rekognition** : Computer Vision API  
* **SNS**: Simple Notification Service, used by Rekognition to provide status of a job (mostly for applications), we will bypass most of its functionality for this lab  


## IAM and Access
AWS Identity and Access Management (IAM) enables you to manage access to AWS services and resources.  
In the next few steps we will add some permissions for our code to run.

1. Create a Role which gives Rekognition permissions to use SNS  
In this step we are just creating the Role. This new Role will be referenced in Step 3 below.  

 * In IAM Console select create role
 * In the next screen select Rekognition
 * Click next all the way to the end and give this role a name, such as:  "rekognition_access_to_sns"
<br>

2. Add *RekognitionFullAccess* and *SNSFullAccess* to the Sagemaker Execution Role  
![](smexecutionrole.png)

 * Go to the IAM service and click Roles
 * Click the SageMaker Execution Role
 * Click "Attach Policies"
 * In the list of services, search for *Rekognition* and then select *RekognitionFullAccess* 
 * In the list of services, search for *SNS* and then select *SNSFullAccess* 
 * Click "Attach policies"
<br>

![](rekservicerole.png)

3. Attach an inline Policy to the SageMaker Execution Role which allows passing the Role created in Step 2 to Rekognition
 * Click "Add inline policy"
 * Select the JSON tab
 * Copy and paste the following Policy into the text box
 * Replace the "Resource" with the ARN of the Role you created in Step 1 above
 * Click "Review policy"
 * Type a name for this inline Policy and click "Attach policy"
```yaml
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "rekoginitRekogPass",
            "Effect": "Allow",
            "Action": "iam:PassRole",
            "Resource": "arn:aws:iam::xxxxxxxx:role/rekognition_access_to_sns"
        }
    ]
}
```




#### Import some standard libraries to be used by our python code


In [ ]:
import boto3
import json
import sys
import time
import os.path

#### Let's declare some variable of where our video file is and its name etc ..

In [ ]:
# the name of the S3 bucket where the video to be analyzed will be stored
bucket = '<ENTER YOUR BUCKET NAME HERE>'

# the location of the video file to be analyzied
video_url = "https://ia801703.us.archive.org/1/items/2020-01-06_videos-WashingtonDC-Propublica/bjXvAoK0Gu5Fcvt.mp4"

# just the name of the video file
video = os.path.basename(video_url)

# the ARN of the Role created in Step 2: IAM and Access
RekognitionSnsRoleArn = '<ENTER YOUR REKOGNITION ROLE ARN HERE>'


In [ ]:
# get a copy of the video from the source location
!curl $video_url --output $video
!aws s3 cp $video "s3://$bucket/"

## Initiate boto3 clients for Rekognition and SNS
The boto3 library is the AWS SDK for Python which makes it easy to integrate your Python application, library, or script with AWS services. 

In [ ]:
rek = boto3.client('rekognition')
sns = boto3.client('sns')

### We need a SNS Topic to track the progress of Rekognition Video 
A message is sent to the SNS topic when the video analysis is complete.


In [ ]:
response = sns.create_topic(Name='rekVideoStatus')

In [ ]:
response


In [ ]:
topicArn = response['TopicArn']

### 1. Rekgonition Video Text Detection API
Amazon Rekognition text detection can detect text in videos, converting the detected text into machine-readable text. It can read skewed and distorted text to capture information like store names, forced narratives overlaid on media, street signs, and text on product packaging.  

The *start_text_detection* API call initiates the video analysis.

In [ ]:
response = rek.start_text_detection(Video={'S3Object': {'Bucket': bucket, 'Name': video}},
            NotificationChannel={'RoleArn': RekognitionSnsRoleArn , 'SNSTopicArn': topicArn})

In [ ]:
jobId = response['JobId']

#### Results

When you execute the cell below, if the job is still in progress we will see JobStatus as "IN PROGRESS".  
If it is complete, JobStatus will be set to "SUCCEEDED" and we will get the first MaxResults (10 in this example) results.  
For this exercise, we can invoke the API call repeatedly using Ctrl-Enter to run this cell.



In [ ]:
rek.get_text_detection(
    JobId=jobId,
    MaxResults=10
)

#### Retrieving all the results...
If we want more (or all) the results, we can use a while Loop until 'NextToken' is empty, like so:

```yaml
results=[]
paginationToken = ''
finished = False

while finished == False:
    '''if you want to just read it into a list'''
    temp = rek.get_text_detection(
        JobId = jobId,
        MaxResults = 10,
        NextToken = paginationToken
    )
    results += temp['TextDetections']
    paginationToken = temp['NextToken']
    if len(paginationToken) == 0:
        finished = True
    
'''if you want to write to file'''
with open(filename, 'a+') as f:
    f.write("parsed results here")
        
'''of you can use a csv writer if you want a csv file'''
```
    

### 2. Rekognition Video Content Moderation API
With Amazon Rekognition you can detect explicit adult or suggestive content, violence, weapons, drugs, tobacco, alcohol, hate symbols, gambling, disturbing content, and rude gestures in both images and videos, and get back a confidence score for each detected label.  

All we change in the code is the call to *start_content_moderation*, quite simple


In [ ]:
response = rek.start_content_moderation(Video={'S3Object': {'Bucket': bucket, 'Name': video}},
            NotificationChannel={'RoleArn': RekognitionSnsRoleArn, 'SNSTopicArn': topicArn})

In [ ]:
jobId = response['JobId']

In [ ]:
rek.get_content_moderation(
    JobId=jobId,
    MaxResults=10,
)

### 3. Rekognition Video Label Detection
With Amazon Rekognition, you can identify thousands of objects (such as bike, telephone, building), and scenes (such as parking lot, beach, city). You can also identify specific activities such as "delivering a package" or "playing soccer".  

All we change here is the API call to *start_object_detection*, again - quite simple

In [ ]:
response = rek.start_label_detection(Video={'S3Object': {'Bucket': bucket, 'Name': video}},
            NotificationChannel={'RoleArn': RekognitionSnsRoleArn, 'SNSTopicArn': topicArn})

In [ ]:
jobId = response['JobId']

In [ ]:
rek.get_label_detection(
    JobId = jobId,
    MaxResults = 10
)